In [27]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, roc_auc_score

### 1. importing dataset
* replace missing with ?

In [28]:
df = pd.read_excel('10k_diabetes.xlsx')
df = df.fillna('?')
df.shape

(10000, 51)

### 2. removing categorical columns
* multiple values (>20) - diag_3, diag_3_desc, diag_2, diag_2_desc, diag_1, diag_1_desc, discharge_disposition_id and medical_specialty
* single value (=1) - acetohexamide, citoglipton, examide, glimepiride.pioglitazone, metformin.pioglitazone, metformin.rosiglitazone and troglitazone
* highly skewed variable (tail count<30)- chlorpropamide, tolbutamide, miglitol, tolazamide and glipizide.metformin

In [29]:
df = df.drop(['diag_3','diag_3_desc','diag_2','diag_2_desc','diag_1','diag_1_desc',
              'discharge_disposition_id','medical_specialty'], axis=1)
df = df.drop(['acetohexamide','citoglipton','examide','glimepiride.pioglitazone',
              'metformin.pioglitazone','metformin.rosiglitazone','troglitazone'], axis=1)
df = df.drop(['chlorpropamide','tolbutamide','miglitol','tolazamide','glipizide.metformin'], axis=1)

df['age'] = np.where(df['age'].isin(['[10-20)','[0-10)']), 'Others', df['age'])
df['weight'] = np.where(df['weight'].isin(['[125-150)','[25-50)','[0-25)','[150-175)']), 'Others', df['weight'])
df['admission_type_id'] = np.where(df['admission_type_id'].isin(['Not Mapped','Newborn']), 'Others', df['admission_type_id'])
df['admission_source_id'] = np.where(df['admission_source_id'].isin(['HMO Referral','Not Mapped','Not Available','Court/Law Enforcement']), 'Others', df['admission_source_id'])
df['payer_code'] = np.where(df['payer_code'].isin(['WC','SI','CH','OT']), 'Others', df['payer_code'])
df['repaglinide'] = np.where(df['repaglinide']=='No', df['repaglinide'], 'Others')
df['nateglinide'] = np.where(df['nateglinide']=='No', df['nateglinide'], 'Others')
df['glimepiride'] = np.where(df['glimepiride'].isin(['Up','Down']), 'Others', df['glimepiride'])
df['pioglitazone'] = np.where(df['pioglitazone'].isin(['Up','Down']), 'Others', df['pioglitazone'])
df['rosiglitazone'] = np.where(df['rosiglitazone'].isin(['Up','Down']), 'Others', df['rosiglitazone'])
df['acarbose'] = np.where(df['acarbose']=='No', df['acarbose'], 'Others')
df['glyburide.metformin'] = np.where(df['glyburide.metformin']=='No', df['glyburide.metformin'], 'Others')
df.shape

(10000, 31)

### 3. removing numerical columns
* num_procedures has opposite trend (corr is -1)
* there are no missings in numerical variables
* all numerical variables have positive relationship with readmitted

In [30]:
df = df.drop(['num_procedures'], axis=1)
df['readmitted'] = np.where(df['readmitted']==False, 0, 1)
df[['readmitted','time_in_hospital','num_lab_procedures','num_medications',
    'number_outpatient','number_emergency','number_inpatient','number_diagnoses']].corr()

,readmitted,time_in_hospital,num_lab_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
readmitted,1.000000,0.051516,0.081649,0.044450,0.073063,0.076183,0.194647,0.155551
time_in_hospital,0.051516,1.000000,0.291957,0.475244,-0.032630,-0.007487,0.080876,0.258570
num_lab_procedures,0.081649,0.291957,1.000000,0.236220,-0.028577,-0.001317,0.087109,0.151874
num_medications,0.044450,0.475244,0.236220,1.000000,0.025709,0.006526,0.044105,0.251479
number_outpatient,0.073063,-0.032630,-0.028577,0.025709,1.000000,0.082542,0.075716,0.095477
number_emergency,0.076183,-0.007487,-0.001317,0.006526,0.082542,1.000000,0.187034,0.032680
number_inpatient,0.194647,0.080876,0.087109,0.044105,0.075716,0.187034,1.000000,0.124547
number_diagnoses,0.155551,0.258570,0.151874,0.251479,0.095477,0.032680,0.124547,1.000000


### 4. removing missing columns
* % missing > 50% 
* dropped weight, max_glu_serum, A1Cresult and payer_code

In [31]:
df = df.drop(['weight','max_glu_serum','A1Cresult','payer_code'], axis=1)
df.shape

(10000, 26)

In [32]:
for i in df.columns:
    print(i,' : ',df[df[i]=='?'].shape)

race  :  (221, 26)
gender  :  (0, 26)
age  :  (0, 26)
admission_type_id  :  (721, 26)
admission_source_id  :  (936, 26)
time_in_hospital  :  (0, 26)
num_lab_procedures  :  (0, 26)
num_medications  :  (0, 26)
number_outpatient  :  (0, 26)
number_emergency  :  (0, 26)
number_inpatient  :  (0, 26)
number_diagnoses  :  (0, 26)
metformin  :  (0, 26)
repaglinide  :  (0, 26)
nateglinide  :  (0, 26)
glimepiride  :  (0, 26)
glipizide  :  (0, 26)
glyburide  :  (0, 26)
pioglitazone  :  (0, 26)
rosiglitazone  :  (0, 26)
acarbose  :  (0, 26)
insulin  :  (0, 26)
glyburide.metformin  :  (0, 26)
change  :  (0, 26)
diabetesMed  :  (0, 26)
readmitted  :  (0, 26)


### 5. exporting dataset
* there are 7 numerical variables
* there are 18 categorical variables
* there are 1 target variable - readmitted
* also looking at univariate

In [33]:
df = df[['race','gender','age','admission_type_id','admission_source_id','metformin','repaglinide',
         'nateglinide','glimepiride','glipizide','glyburide','pioglitazone','rosiglitazone','acarbose',
         'insulin','glyburide.metformin','change','diabetesMed','time_in_hospital','num_lab_procedures',
         'num_medications','number_outpatient','number_emergency','number_inpatient','number_diagnoses',
         'readmitted']]
df.to_csv('01 data prep.csv', index=False)
df.shape

(10000, 26)

In [34]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
race,10000,6,Caucasian,7359,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gender,10000,2,Female,5398,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age,10000,9,[70-80),2595,NaN,NaN,NaN,NaN,NaN,NaN,NaN
admission_type_id,10000,6,Emergency,4905,NaN,NaN,NaN,NaN,NaN,NaN,NaN
admission_source_id,10000,8,Emergency Room,4940,NaN,NaN,NaN,NaN,NaN,NaN,NaN
metformin,10000,4,No,8011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
repaglinide,10000,2,No,9870,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nateglinide,10000,2,No,9949,NaN,NaN,NaN,NaN,NaN,NaN,NaN
glimepiride,10000,3,No,9509,NaN,NaN,NaN,NaN,NaN,NaN,NaN
glipizide,10000,4,No,8696,NaN,NaN,NaN,NaN,NaN,NaN,NaN
